In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
IDS2017URL = "/gdrive/MyDrive/IDS2017/IDS2017.csv"

In [3]:
import torch
import pandas as pd
import tensorflow as tf
import numpy as np
import torch.nn as nn
from sklearn.metrics import f1_score
from numpy import vstack
from numpy import argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder

In [4]:
import torch
import pandas as pd
import tensorflow as tf
import numpy as np
import torch.nn as nn
import glob
import time
import os
from os.path import join
from numpy import vstack
from numpy import argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
from torchvision import transforms

In [5]:
class Classifier:
    def __init__(self,method,input_dim,num_classes,num_epochs,batch_size=100,lr=1e-3,reg=1e-5,runs_dir=None):
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = lr
        self.reg= reg
        self.runs_dir = runs_dir
        #self.device = 'cuda'

        #self.model = nn.Linear(self.input_size, self.num_classes).to(self.device)
        if method=='softmax':
            self.device = torch.device('cuda:1')
            self.model = Softmax(input_dim,num_classes=num_classes, device=self.device)
        elif method=='cnn2':
            self.device = torch.device('cuda:2')
            self.model = CNN2(input_dim,num_classes=num_classes,device=self.device)        
        elif method=='cnn5':
            self.device = torch.device('cuda:0')
            self.model = CNN5(input_dim,num_classes=num_classes,device=self.device)        

        elif method=='nn3':
            self.device = torch.device('cuda:0')
            self.model = Net3(input_dim,num_classes=num_classes,device=self.device)        
        elif method=='nn5':
            self.device = torch.device('cuda:0')
            self.model = Net5(input_dim,num_classes=num_classes,device=self.device)        
        else:
            print('There is no such classifier')
            exit()
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=self.learning_rate,betas=(0.9,0.99),eps=1e-08, weight_decay=self.reg, amsgrad=False)

    def fit(self,X,Y):
        sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=2)
        for dev_index, val_index in sss.split(X, Y): # runs only once
                X_dev = X[dev_index]
                Y_dev = Y[dev_index]
                X_val = X[val_index]
                Y_val = Y[val_index]  
        writer = SummaryWriter(self.runs_dir) 

        tensor_x = torch.stack([torch.Tensor(i) for i in X_dev]).to(self.device)
        tensor_y = torch.LongTensor(Y_dev).to(self.device) # checked working correctly

        dataset = utils.TensorDataset(tensor_x,tensor_y)        
        train_loader = utils.DataLoader(dataset,batch_size=self.batch_size) 
        N = tensor_x.shape[0]

        num_epochs = self.num_epochs

        model  = self.model
        best_acc = None
        best_epoch = None

        filepath = join(self.runs_dir,'checkpoint.pth')
        if os.path.isfile(filepath):
            checkpoint = self.load_checkpoint(filepath)
            best_epoch = checkpoint['epoch']
            best_batch = checkpoint['batch']
            self.model.load_state_dict(checkpoint['state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer'])
            pred = self.predict(X_val)
            best_acc = metrics.balanced_accuracy_score(Y_val,pred)*100
            resume_epoch = best_epoch+1  
            resume_batch = best_batch+1
        else:
            resume_epoch = 0
            resume_batch = 0
            best_acc = -1
            best_epoch = 0

        no_improvement = 0
        print("best epoch {}, best batch {}".format(resume_epoch,resume_batch))
        print("bst acc ", best_acc)
        for epoch in range(resume_epoch,num_epochs):
            for i,(xi,yi) in enumerate(train_loader):
                if epoch==resume_epoch and i<resume_batch:
                    continue    
                outputs = model(xi)
                loss = self.criterion(outputs,yi)

                loss.requires_grad
                seen_so_far = self.batch_size*(epoch*len(train_loader)+i+1) # fixes issues with different batch size
                writer.add_scalar('Loss/train',loss.item(),seen_so_far)
                #batckward, optimize
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                if (seen_so_far/self.batch_size)%50==0:
                    pred = self.predict(X_val)
                    balanced_acc = metrics.balanced_accuracy_score(Y_val,pred)*100
                    if balanced_acc > best_acc:
                        best_acc = balanced_acc
                        best_epoch = epoch
                        checkpoint = {
                        'state_dict': model.state_dict(),
                        'optimizer' : self.optimizer.state_dict(),
                        'epoch':epoch,
                        'batch': i,
                        'batch_size': self.batch_size
                        }
                        self.save(checkpoint)
                        no_improvement =0
                    else:
                        no_improvement+=1
                        if no_improvement>=100:
                            print("no improvement in accuracy for 10 iterations")
                            return
                        
                    writer.add_scalar('Accuracy/Balanced Val',balanced_acc,seen_so_far)

                    acc = metrics.accuracy_score(Y_val,pred)*100
                    writer.add_scalar('Accuracy/Val',acc,seen_so_far)
                    print ('Accuracy/Val', acc)    
                    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                                               .format(epoch+1, num_epochs, i+1, len(Y_dev)//self.batch_size, loss.item()))
        writer.close()

    def predict(self,x,eval_mode=False):
        tensor_x = torch.stack([torch.Tensor(i) for i in x]).to(self.device)
        bs = self.batch_size
        num_batch = x.shape[0]//bs +1*(x.shape[0]%bs!=0)

        pred = torch.zeros(0,dtype=torch.int64).to(self.device)
        
        if eval_mode:
            model = self.load_model()
        else:
            model = self.model
        model.eval()        
        
        with torch.no_grad():
            for i in range(num_batch):
                xi = tensor_x[i*bs:(i+1)*bs]
                outputs = model(xi)
                _, predi = torch.max(outputs.data,1)
                pred = torch.cat((pred,predi))

        return pred.cpu().numpy()


    def save(self,checkpoint):
        path = join(self.runs_dir,'checkpoint.pth')
        torch.save(checkpoint,path)

    
    def load_checkpoint(self,filepath):
        if os.path.isfile(filepath):
            checkpoint = torch.load(filepath)
            print("Loaded {} model trained with batch_size = {}, seen {} epochs and {} mini batches".
                format(self.runs_dir,checkpoint['batch_size'],checkpoint['epoch'],checkpoint['batch'])) 
            return checkpoint
        else:
            return None
        
            
    def load_model(self,inference_mode=True):
        filepath = join(self.runs_dir,'checkpoint.pth')
        checkpoint = self.load_checkpoint(filepath)
        
        model = self.model
        model.load_state_dict(checkpoint['state_dict'])
        
        if inference_mode:
            for parameter in model.parameters():
                parameter.requires_grad = False
            model.eval()
        return model

In [6]:
class CNN5(nn.Module):
    def __init__(self,input_dim,num_classes,device):
        super(CNN5, self).__init__()
        # kernel
        self.input_dim = input_dim
        self.num_classes = num_classes

        
        conv_layers = []
        self.conv1 = nn.Conv1d(in_channels=1,out_channels=64,kernel_size=3,padding=1)
        constant_(self.conv1.weight, 0.1)
        constant_(self.conv1.bias, 0.1)

        conv_layers.append(self.conv1) # ;input_dim,64
        conv_layers.append(nn.BatchNorm1d(64))
        conv_layers.append(nn.ReLU(True))

        self.conv2 = nn.Conv1d(in_channels=64,out_channels=128,kernel_size=3,padding=1)
        constant_(self.conv2.weight, 0.1)
        constant_(self.conv2.bias, 0.1)
        conv_layers.append(self.conv2) #(input_dim,128)
        conv_layers.append(nn.BatchNorm1d(128))
        conv_layers.append(nn.ReLU(True))

        self.conv3 = nn.Conv1d(in_channels=128,out_channels=256,kernel_size=3,padding=1)
        constant_(self.conv3.weight, 0.1)
        constant_(self.conv3.bias, 0.1)
        conv_layers.append(self.conv3) #(input_dim,128)
        conv_layers.append(nn.BatchNorm1d(256))
        conv_layers.append(nn.ReLU(True))

        self.conv4 = nn.Conv1d(in_channels=256,out_channels=256,kernel_size=3,padding=1)
        constant_(self.conv4.weight, 0.1)
        constant_(self.conv4.bias, 0.1)
        conv_layers.append(self.conv4) #(input_dim,128)
        conv_layers.append(nn.BatchNorm1d(256))
        conv_layers.append(nn.ReLU(True))

        self.conv5 = nn.Conv1d(in_channels=256,out_channels=128,kernel_size=3,padding=1)
        constant_(self.conv5.weight, 0.1)
        constant_(self.conv5.bias, 0.1)
        conv_layers.append(self.conv5) #(input_dim,128)
        conv_layers.append(nn.BatchNorm1d(128))
        conv_layers.append(nn.ReLU(True))
        
        self.conv = nn.Sequential(*conv_layers).to(device)

        fc_layers = []
        self.l1 = nn.Linear(input_dim*128,num_classes)
        constant_(self.l1.weight, 0.1)
        constant_(self.l1.bias, 0.1)
        fc_layers.append(self.l1)
        self.classifier = nn.Sequential(*fc_layers).to(device)

    def forward(self, x):
        batch_size, D = x.shape
        x = x.view(batch_size,1,D)

        x = self.conv(x)
        x = torch.flatten(x,1)
        x = self.classifier(x)
        return x



In [7]:
def ensure_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

def getClassifier(args,runs_dir=None):
    (method,optim,lr,reg,batch_size,input_dim,num_class,num_epochs) = args
    if runs_dir is not None:
        ensure_dir(runs_dir)
    device = torch.device('cuda:0')
    model = Classifier(method,input_dim,num_class,lr=lr,reg=reg,num_epochs=num_epochs,
                        batch_size=batch_size,runs_dir=runs_dir)
    return model

In [8]:
#balance dataset
def balance_data(X,y,seed):
  np.random.seed(SEED)
  unique,counts = np.unique(y,return_counts=True)
  #unique
  #array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
  #counts
  #array([2357511,    1956,   41834,   10293,  230124,    5499,    5796, 7935,      11,      36,  158804,    5897,    2180])
  mean_samples_per_class = int(round(np.mean(counts)))
  N,D = X.shape
  new_X = np.empty((0,D))
  new_y = np.empty((0),dtype=int)
  for i,c in enumerate(unique):
      temp_x = X[y==c]
      indices = np.random.choice(temp_x.shape[0],mean_samples_per_class) # gets `mean_samples_per_class` indices of class `c`
      new_X = np.concatenate((new_X,temp_x[indices]),axis=0) # now we put new data into new_X 
      temp_y = np.ones(mean_samples_per_class,dtype=int)*c
      new_y = np.concatenate((new_y,temp_y),axis=0)
      indices = np.arange(new_y.shape[0])
      np.random.shuffle(indices)
      new_X =  new_X[indices,:]
      new_y = new_y[indices]
  return (new_X, new_y)

In [9]:
data = pd.read_csv(IDS2017URL)
data.loc[data['Label'].isin([ 'Web Attack - Sql Injection','Web Attack - XSS', 'Web Attack - Brute Force']), 'Label'] = 'Web Attack'
data = data[['Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Flow IAT Max', 'Bwd Packet Length Std', 'Fwd Packet Length Max', 'Flow Bytes/s', 'Total Length of Bwd Packets', 'Fwd Packet Length Mean', 'Flow Duration', 'Flow IAT Min', 'Total Length of Fwd Packets', 'Flow IAT Mean', 'Total Backward Packets', 'Bwd Packet Length Max', 'Flow Packets/s', 'Flow IAT Std', 'Fwd IAT Total', 'Bwd Packet Length Min', 'Fwd Packet Length Min', 'Label']]
data = data.replace([-np.inf, np.inf], np.nan)
data = data.dropna()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (84) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
num_records, num_features = data.shape

In [11]:
X = data.iloc[:,:-1].astype(float).apply(pd.to_numeric).apply(lambda x: (x-np.mean(x))/np.std(x)).values

In [12]:
le = LabelEncoder()
le.fit(["BENIGN", "DoS Hulk", "PortScan", "DDoS", "DoS GoldenEye","FTP-Patator","SSH-Patator","DoS slowloris","DoS Slowhttptest","Web Attack","Bot","Infiltration","Heartbleed"])
y = le.transform(data['Label'].values)

In [13]:
X

array([[-0.28068619, -0.24519349, -0.37563189, ..., -0.43161033,
        -0.59652323, -0.2109059 ],
       [-0.28068619, -0.24519349, -0.37563201, ..., -0.43161042,
        -0.59652323, -0.2109059 ],
       [-0.28068619, -0.24519349, -0.37563201, ..., -0.43161042,
        -0.59652323, -0.2109059 ],
       ...,
       [-0.31291423, -0.24519349, -0.37562735, ..., -0.43161044,
        -0.59652323, -0.31031719],
       [-0.04792816,  0.05854422, -0.37172996, ..., -0.42875763,
        -0.59652323, -0.31031719],
       [-0.31291423, -0.24519349, -0.37563005, ..., -0.43161044,
        -0.59652323, -0.31031719]])

In [14]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [15]:
#split train set and test set
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics
K=5
SEED=2

skf = StratifiedKFold(n_splits=K,random_state=SEED)
for fold_index, (train_index,test_index) in enumerate(skf.split(X,y)):# runs only once   
        X_train = X[train_index]
        y_train = y[train_index]
        X_test = X[test_index]
        y_test = y[test_index]
        break

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [16]:
batch_size = 5*1024
optim = 'Adam'
input_dim = X_train.shape[1]
num_class = 13

num_epochs = 20
lr = 1e-2
reg = 1e-3

#
accuracies = {}
best_model = None
best_acc = -1
architecture = '1D-CNN_conv_5_fc_1'
run_number = 5
method='cnn5'

classifier_args = (method,optim,lr,reg,batch_size,input_dim,num_class,num_epochs)

runs_dir = "gdrive/MyDrive/IDS2017/runs/"

In [17]:
X_train = X_train.astype(float)
y_train = y_train.astype(int)
#shuffle
np.random.seed(SEED)
p = np.random.permutation(len(y_train))
X_train = X_train[p]
y_train = y_train[p]

In [18]:
X_train

array([[-0.14640271, -0.24519349, -0.37562449, ..., -0.43161036,
         1.71180006,  0.2033078 ],
       [-0.18937343, -0.24519349, -0.37562547, ..., -0.43161036,
        -0.26261483,  0.07075942],
       [-0.06583263, -0.24519349, -0.37437109, ..., -0.43161036,
         0.63748607,  0.45183602],
       ...,
       [-0.03897593, -0.24519349, -0.37562188, ..., -0.43161036,
         0.37616646,  0.53467876],
       [-0.26636262, -0.20871133, -0.37138166, ..., -0.43159327,
        -0.59652323, -0.31031719],
       [-0.31291423, -0.24519349, -0.37563136, ..., -0.43161044,
        -0.50941669, -0.31031719]])

In [19]:
X_train.shape

(2262300, 19)

In [20]:
X_train,y_train = balance_data(X_train,y_train,seed=SEED)

In [21]:
X_train

array([[-3.12914226e-01, -2.45193488e-01, -3.75632012e-01, ...,
        -4.31610415e-01, -5.96523229e-01, -3.10317193e-01],
       [-2.80686191e-01, -2.45193488e-01, -3.75187349e-01, ...,
        -4.31282904e-01, -5.96523229e-01, -2.10905903e-01],
       [ 1.65134957e-01,  3.75743014e-01,  4.07876875e+00, ...,
         2.81343648e+00, -5.96523229e-01, -3.10317193e-01],
       ...,
       [-2.14389052e-03,  2.98937705e-01, -9.57135423e-02, ...,
        -2.27473474e-01, -5.96523229e-01, -3.10317193e-01],
       [-2.75314852e-01, -2.10004186e-01, -3.75625964e-01, ...,
        -4.31603836e-01, -5.96523229e-01, -3.10317193e-01],
       [-2.75911667e-01, -2.26346732e-01,  3.66193492e+00, ...,
         2.57526790e+00, -5.96523229e-01, -3.10317193e-01]])

In [22]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as utils
from torch.utils.tensorboard import SummaryWriter
import os
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics
from torch.nn.init import xavier_uniform_
from torch.nn.init import constant_
from os.path import join

In [23]:
tick = time.time()
clf = getClassifier(classifier_args,runs_dir)
print(clf.model.state_dict())
clf.fit(X_train,y_train)
pred = clf.predict(X_test,eval_mode=True)
acc = metrics.balanced_accuracy_score(y_test,pred)
print(acc)
if acc >best_acc:
    best_model = clf
    best_acc = acc
accuracies[(lr,reg)]=acc
tock = time.time()
print("Model is trained in {} sec".format(tock-tick))

OrderedDict([('conv1.weight', tensor([[[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        [[0.1000, 0.1000, 0.1000]],

        

In [24]:
# def fit(model,X,Y,batch_size,optimizer):
#   sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=2)
#   for dev_index, val_index in sss.split(X, Y): # runs only once
#       X_dev = X[dev_index]
#       Y_dev = Y[dev_index]
#       X_val = X[val_index]
#       Y_val = Y[val_index]
#   tensor_x = torch.stack([torch.Tensor(i) for i in X_dev]).to(self.device)
#   tensor_y = torch.LongTensor(Y_dev).to(self.device)
#   dataset = utils.TensorDataset(tensor_x,tensor_y)        
#   train_loader = utils.DataLoader(dataset,batch_size=self.batch_size) 
#   N = tensor_x.shape[0] 

#   #
#   num_epochs = 10
#   resume_epoch = 0
#   resume_batch = 0
#   best_acc = -1
#   best_epoch = 0 
#   no_improvement = 0

#    for epoch in range(resume_epoch,num_epochs):
#             for i,(xi,yi) in enumerate(train_loader):
#                 if epoch==resume_epoch and i<resume_batch:
#                     continue                   
#                 outputs = model(xi)
#                 loss = self.criterion(outputs,yi)
#                 loss.requires_grad
#                 seen_so_far = batch_size*(epoch*len(train_loader)+i+1)

#                 soptimizer.zero_grad()
#                 loss.backward()
#                 optimizer.step()

In [25]:
fc = clf.model.state_dict()['classifier.0.weight'].squeeze()

In [26]:
from sklearn.cluster import AgglomerativeClustering

In [27]:
clustering = AgglomerativeClustering(
        linkage='ward',
        n_clusters=2,
        affinity='euclidean',
    ).fit(fc.cpu())

[0"BENIGN", 1"DoS Hulk", 2"PortScan", 3"DDoS", 4"DoS GoldenEye",5"FTP-Patator",6"SSH-Patator",7"DoS slowloris",8"DoS Slowhttptest",9"Web Attack",10"Bot",11"Infiltration",12"Heartbleed"]

In [28]:
#CNN
clustering.children_

array([[ 0,  8],
       [ 9, 13],
       [ 3, 12],
       [ 2, 14],
       [ 5,  6],
       [11, 15],
       [10, 18],
       [ 1, 16],
       [ 4,  7],
       [19, 20],
       [17, 21],
       [22, 23]])